# Cubeviz Demonstration Notebook

This notebook demonstrates the Cubeviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Cubeviz, can be found here: https://jdaviz.readthedocs.io/en/latest/cubeviz/

In [ ]:
from jdaviz import Cubeviz

viz = Cubeviz()
viz.app

In [ ]:
from astropy.utils.data import download_file

# This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz
# but has been modified to correct some inconsistencies in the way units are parsed
fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)
viz.load_data(fn)